In [ ]:
#model1new.pth
import os
import torch
import torch.nn as nn
import torch.optim as optim
import json
import pandas as pd
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from torchvision.models import EfficientNet_B0_Weights
import random

# ✅ GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ 사용 중인 GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

# ✅ 경로 설정
data_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/data/category_data3"
model_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/model"
csv_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/csv"
json_dir = "C:/Users/user/OneDrive/Desktop/Resnet182-real/jsonnew"  # ✅ JSON 저장 경로
os.makedirs(model_dir, exist_ok=True)
os.makedirs(csv_dir, exist_ok=True)
os.makedirs(json_dir, exist_ok=True)  # ✅ JSON 저장 폴더 생성

# ✅ 데이터셋 분할 인덱스 저장 경로
train_indices_path = os.path.join(json_dir, "model1newtrain.json")
val_indices_path = os.path.join(json_dir, "model1newval.json")
test_indices_path = os.path.join(json_dir, "model1newtest.json")

# ✅ 하이퍼파라미터 설정
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 30
LEARNING_RATE = 0.001

# ✅ 데이터 전처리
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ✅ 데이터셋 로드
dataset = datasets.ImageFolder(data_dir, transform=transform)

# ✅ 각 클래스별 이미지 인덱스 분배
class_indices = {cls: [] for cls in dataset.class_to_idx.values()}

for idx, (path, label) in enumerate(dataset.samples):
    class_indices[label].append(idx)

# ✅ 각 클래스별로 7:2:1 비율로 분할 (학습:검증:테스트)
train_indices, val_indices, test_indices = [], [], []

for indices in class_indices.values():
    random.shuffle(indices)  # 랜덤 셔플
    num_total = len(indices)
    num_train = int(num_total * 0.7)
    num_val = int(num_total * 0.2)
    num_test = num_total - num_train - num_val

    train_indices.extend(indices[:num_train])
    val_indices.extend(indices[num_train:num_train+num_val])
    test_indices.extend(indices[num_train+num_val:])

# ✅ 분할된 인덱스를 JSON 파일로 저장
with open(train_indices_path, "w") as f:
    json.dump(train_indices, f)
with open(val_indices_path, "w") as f:
    json.dump(val_indices, f)
with open(test_indices_path, "w") as f:
    json.dump(test_indices, f)

# ✅ 데이터셋 분할
dataset_train = Subset(dataset, train_indices)
dataset_val = Subset(dataset, val_indices)
dataset_test = Subset(dataset, test_indices)

# ✅ 데이터 로더 설정
test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

# ✅ EfficientNet-B0 모델 정의
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(dataset.classes))
model.to(device)

# ✅ 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ✅ 학습 루프
best_valid_accuracy = 0.0
best_epoch = 0
csv_path = os.path.join(csv_dir, "모델1 new 학습기록.csv")
train_losses, valid_losses, train_accuracies, valid_accuracies = [], [], [], []

print("\n📢 Training Started! Logging Every Epoch:\n")

for epoch in range(EPOCHS):
    print(f"\n📢 Epoch [{epoch+1}/{EPOCHS}] 시작")
    model.train()
    train_loss, train_correct = 0, 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()
        total_train += labels.size(0)

    train_loss /= total_train
    train_accuracy = train_correct / total_train
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # ✅ 검증 단계
    model.eval()
    valid_loss, valid_correct = 0, 0
    total_valid = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * inputs.size(0)
            valid_correct += (outputs.argmax(1) == labels).sum().item()
            total_valid += labels.size(0)

    valid_loss /= total_valid
    valid_accuracy = valid_correct / total_valid
    valid_losses.append(valid_loss)
    valid_accuracies.append(valid_accuracy)

# ✅ 최고의 검증 정확도를 기록한 에포크 모델 저장
    print(f"   🎯 Train Accuracy: {train_accuracy:.4f} | 📉 Train Loss: {train_loss:.4f} | "
        f"🎯 Valid Accuracy: {valid_accuracy:.4f} | 📉 Valid Loss: {valid_loss:.4f}")

    if valid_accuracy > best_valid_accuracy:
        best_valid_accuracy = valid_accuracy
        best_epoch = epoch + 1
        best_model_path = os.path.join(model_dir, "model1new.pth")
        torch.save(model.state_dict(), best_model_path)
        print(f"🎯 새로운 최고 모델 저장됨! (Epoch {epoch+1}, Accuracy: {best_valid_accuracy:.4f})")

# ✅ 결과 CSV 저장
df = pd.DataFrame({
    'Epoch': range(1, len(train_losses) + 1),
    'Train Loss': train_losses,
    'Valid Loss': valid_losses,
    'Train Accuracy': train_accuracies,
    'Valid Accuracy': valid_accuracies
})
csv_path = os.path.join(csv_dir, "모델1new.csv")
df.to_csv(csv_path, index=False)
print(f"📄 학습 결과 CSV 저장됨: {csv_path}")

# ✅ 검증 정확도가 가장 높은 에포크의 모든 정보 출력
print(f"✅ 학습 종료! 최고 성능을 기록한 Epoch: {best_epoch}, 🎯 Train Accuracy: {train_accuracies[best_epoch-1]:.4f}, 📉 Train Loss: {train_losses[best_epoch-1]:.4f}, 🎯 Valid Accuracy: {best_valid_accuracy:.4f}, 📉 Valid Loss: {valid_losses[best_epoch-1]:.4f}")


✅ 사용 중인 GPU: Quadro RTX 4000

📢 Training Started! Logging Every Epoch:


📢 Epoch [1/30] 시작
   🎯 Train Accuracy: 0.7751 | 📉 Train Loss: 0.7066 | 🎯 Valid Accuracy: 0.8609 | 📉 Valid Loss: 0.4043
🎯 새로운 최고 모델 저장됨! (Epoch 1, Accuracy: 0.8609)

📢 Epoch [2/30] 시작
   🎯 Train Accuracy: 0.8984 | 📉 Train Loss: 0.3202 | 🎯 Valid Accuracy: 0.8455 | 📉 Valid Loss: 0.5641

📢 Epoch [3/30] 시작
   🎯 Train Accuracy: 0.9305 | 📉 Train Loss: 0.2186 | 🎯 Valid Accuracy: 0.8808 | 📉 Valid Loss: 0.4280
🎯 새로운 최고 모델 저장됨! (Epoch 3, Accuracy: 0.8808)

📢 Epoch [4/30] 시작
   🎯 Train Accuracy: 0.9450 | 📉 Train Loss: 0.1698 | 🎯 Valid Accuracy: 0.8661 | 📉 Valid Loss: 0.5247

📢 Epoch [5/30] 시작
   🎯 Train Accuracy: 0.9513 | 📉 Train Loss: 0.1520 | 🎯 Valid Accuracy: 0.9014 | 📉 Valid Loss: 0.3233
🎯 새로운 최고 모델 저장됨! (Epoch 5, Accuracy: 0.9014)

📢 Epoch [6/30] 시작
   🎯 Train Accuracy: 0.9628 | 📉 Train Loss: 0.1133 | 🎯 Valid Accuracy: 0.8845 | 📉 Valid Loss: 0.4239

📢 Epoch [7/30] 시작
   🎯 Train Accuracy: 0.9626 | 📉 Train Loss: 0.1136 | 🎯